In [1]:
import pandas as pd
import json
import requests
import time
from requests import get

In [2]:
df=pd.read_excel('products.xlsx',na_values='nope')

In [3]:
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,NaN,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


In [4]:
df=df.dropna(axis=0,subset=['ip_address'])

In [5]:
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33 MGA,Ingredients: sugar and milk


In [6]:
getprice = df["price"].str.split(" ",n = 1, expand = True) 
df.price = getprice[0]
df["currency"] = getprice[1]

In [7]:
df.head()

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",None
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA


In [8]:
#ips=df['ip_address']

In [9]:
def Currency_IP(ip):
    api_url = 'http://free.ipwhois.io/json/'
    try:
        data=[]
        url = api_url + ip
        response = requests.get(url)
        time.sleep(1.0)
        data = json.loads(response.text)
        time.sleep(1.0)
        cur=(data['currency_code'])
        return cur
    except:
        return None
  
    
    
    

In [10]:
df.currency = df.ip_address.apply(Currency_IP)

In [11]:
df.head()

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,MYR
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MYR


In [12]:
df.dropna(subset=['currency'], inplace=True)

In [13]:
df.head()

,username,ip_address,product,price,infos,currency
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,MYR
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MYR
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,JPY


In [14]:
url1="http://data.fixer.io/api/latest?access_key=581be85d2d399af866fb994f48f0911e&format=1"
fx = requests.get(url1).json()["rates"]
df.price = pd.to_numeric(df.price)
df.price = df.apply(lambda k: k['price'] / fx[k['currency']], axis=1)



In [15]:
df.head()

,username,ip_address,product,price,infos,currency
2,abromet2,240.177.79.234,Island Oasis - Lemonade,65.554368,Ingredients: mustard and fish,MYR
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",314.231741,Contains gluten,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,7.853034,"May contain sugar, egg and fish",JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,75.918026,Ingredients: sugar and milk,MYR
6,afairholme6,127.197.204.119,Chicken - Tenderloin,4.008661,May contain sugar,JPY


In [16]:
contains = df.infos \
.str.lower()\
.str.replace('contains','')\
.str.replace('may','')\
.str.replace('contain','')\
.str.replace('and','')\
.str.replace('ingredient','')\
.str.replace(':','')\
.str.replace(',','')\
.str.replace(';','')\
.str.replace('.','')\
.str.replace('/','')\
.str.replace('(','')\
.str.replace(')','')\
.str.replace("ingredients",'')\
.str.split(" ")

In [17]:
dummies = pd.get_dummies(contains.explode())
dummies = dummies.groupby(dummies.index).sum()
dummies = dummies.drop(columns="s")
dummies=dummies.drop(dummies.columns[0], axis=1)
dummies = dummies.astype('bool')

In [18]:
dummies

,egg,fish,gluten,milk,mustard,peanut,soja,sugar
2,False,True,False,False,True,False,False,False
3,False,False,True,False,False,False,False,False
4,True,True,False,False,False,False,False,True
5,False,False,False,True,False,False,False,True
6,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...
192,False,False,False,False,False,False,False,True
193,True,True,False,False,False,False,False,True
195,False,True,True,True,True,False,False,True
197,False,False,True,False,False,False,True,False


In [19]:
df = df.join(dummies)

In [20]:
df

,username,ip_address,product,price,infos,currency,egg,fish,gluten,milk,mustard,peanut,soja,sugar
2,abromet2,240.177.79.234,Island Oasis - Lemonade,65.554368,Ingredients: mustard and fish,MYR,False,True,False,False,True,False,False,False
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",314.231741,Contains gluten,USD,False,False,True,False,False,False,False,False
4,mbuckney4,58.90.204.239,Radish - Pickled,7.853034,"May contain sugar, egg and fish",JPY,True,True,False,False,False,False,False,True
5,bsnozzwell5,226.52.32.70,Oil - Sesame,75.918026,Ingredients: sugar and milk,MYR,False,False,False,True,False,False,False,True
6,afairholme6,127.197.204.119,Chicken - Tenderloin,4.008661,May contain sugar,JPY,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,ioxtoby5c,224.1.159.40,Brandy Cherry - Mcguinness,151.103289,Ingredients: sugar,MYR,False,False,False,False,False,False,False,True
193,tliddyard5d,93.22.46.177,Shrimp - Black Tiger 6 - 8,300.030000,"Ingredients: sugar, egg and fish",EUR,True,True,False,False,False,False,False,True
195,ccannop5f,174.5.73.129,Nantucket Orange Juice,489.403348,"Contains gluten, mustard, sugar, milk and fish",CAD,False,True,True,True,True,False,False,True
197,iollarenshaw5h,206.30.25.226,"Juice - Apple, 341 Ml",401.299465,May contain gluten and soja,USD,False,False,True,False,False,False,True,False
